In [3]:
import pandas as pd
import numpy as np
import json
from ast import literal_eval
from datetime import datetime


In [27]:
df = pd.read_csv('../data/recipes/RAW_recipes.csv')
df = df.dropna()
df['tags'] = df['tags'].apply(lambda x: literal_eval(str(x)))
df['ingredients'] = df['ingredients'].apply(lambda x: literal_eval(str(x)))
df['steps'] = df['steps'].apply(lambda x: literal_eval(str(x)))

season_df = pd.read_csv('../data/seasonality.csv')
season_df['foods'] = season_df['foods'].apply(lambda x: literal_eval(str(x)))

In [30]:
now = datetime.now()
month_array = ['january', 'february', 'march', 'april', 'may', 'june', 'july',\
       'august', 'september', 'october', 'november', 'december']

month = month_array[now.month-1]
period = 'early' if now.day <= 15 else 'late'

In [31]:
seasonal_and_local = season_df.query('month=="{}" & period=="{}"'.format(month, period))


In [32]:
seasonal_and_local.head()

,state,month,period,foods
7,alaska,april,late,"[Chives, Mushrooms, Rosemary, Sage, Sprouts, T..."
31,alabama,april,late,"[Arugula, Asparagus, Beets, Bok Choy, Cabbage,..."
55,arkansas,april,late,"[Arugula, Asparagus, Beets, Black Eyed Peas, C..."
79,arizona,april,late,"[Arugula, Asparagus, Cabbage, Carrots, Caulifl..."
103,california,april,late,"[Artichokes, Arugula, Asparagus, Avocados, Bee..."


In [66]:
seasonal_set = set()
for food_array in seasonal_and_local.foods:
    for food_item in food_array:
        curr_word = food_item.lower()
        seasonal_set.add(curr_word)

selected_state = 'arizona'
snl_array = seasonal_and_local.query('state=="{}"'.format(selected_state)).iloc[0].foods
snl_array = [food.lower() for food in snl_array]

exclusive_seasonal = [x for x in seasonal_set if x not in snl_array]


In [45]:
full_food_set = set()
count = 0
for food_array in season_df.foods:
    for food_item in food_array:
        curr_word = food_item.lower()
        full_food_set.add(curr_word)
        count += 1
print(len(full_food_set), count)

138 45160


Some notes:
- Most foods seem to be seasonal right now (114/138) if we're looking at all states (April 2021)
- We might need to think of a weighting for different foods (apples vs brocolli vs oregano) -> herbs may not count as much since it's everywhere
- We could also ask an expert for what to look at (how food is transported between states, what to weigh based on how much or where it's produced)
- We could do a relative score based on number of ingredients as well

## Getting sustainability score for recipes

In [50]:
recipes_df = df.iloc[0:25]

In [52]:
recipes_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"[time-to-make, course, preparation, main-dish,...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"[60-minutes-or-less, time-to-make, course, mai...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,[place potatoes in a large pot of lightly salt...,"this is a super easy, great tasting, make ahea...","[spreadable cheese with garlic and herbs, new ...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"[weeknight, time-to-make, course, main-ingredi...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,"[mix all ingredients& boil for 2 1 / 2 hours ,...",my dh's amish mother raised him on this recipe...,"[tomato juice, apple cider vinegar, sugar, sal...",8


In [67]:
def sus_score(row):
    seasonal_match = [x for x in row.ingredients if x in exclusive_seasonal]
    snl_match = [x for x in row.ingredients if x in snl_array]

    row['seasonal'] = seasonal_match
    row['snl'] = snl_match
    row['sus_score'] = len(seasonal_match) + (len(snl_match) * 2)

    # TODO: Fix weightage of foods based on better factors and smarter reasoning...
    return row

recipes_df = recipes_df.apply(lambda x: sus_score(x), axis=1)

In [68]:
recipes_df

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,sus_score,seasonal,snl
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",7,1,[winter squash],[]
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",6,0,[],[]
2,all in the kitchen chili,112140,130,196586,2005-02-25,"[time-to-make, course, preparation, main-dish,...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...",13,2,[],[lettuce]
3,alouette potatoes,59389,45,68585,2003-04-14,"[60-minutes-or-less, time-to-make, course, mai...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,[place potatoes in a large pot of lightly salt...,"this is a super easy, great tasting, make ahea...","[spreadable cheese with garlic and herbs, new ...",11,5,[shallots],"[parsley, tarragon]"
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"[weeknight, time-to-make, course, main-ingredi...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,"[mix all ingredients& boil for 2 1 / 2 hours ,...",my dh's amish mother raised him on this recipe...,"[tomato juice, apple cider vinegar, sugar, sal...",8,0,[],[]
6,aww marinated olives,25274,15,21730,2002-04-14,"[15-minutes-or-less, time-to-make, course, mai...","[380.7, 53.0, 7.0, 24.0, 6.0, 24.0, 6.0]",4,[toast the fennel seeds and lightly crush them...,my italian mil was thoroughly impressed by my ...,"[fennel seeds, green olives, ripe olives, garl...",9,2,[],[garlic]
7,backyard style barbecued ribs,67888,120,10404,2003-07-30,"[weeknight, time-to-make, course, main-ingredi...","[1109.5, 83.0, 378.0, 275.0, 96.0, 86.0, 36.0]",10,[in a medium saucepan combine all the ingredie...,this recipe is posted by request and was origi...,"[pork spareribs, soy sauce, fresh garlic, fres...",22,0,[],[]
9,beat this banana bread,75452,70,15892,2003-11-04,"[weeknight, time-to-make, course, main-ingredi...","[2669.3, 160.0, 976.0, 107.0, 62.0, 310.0, 138.0]",12,"[preheat oven to 350 degrees, butter two 9x5"" ...",from ann hodgman's,"[sugar, unsalted butter, bananas, eggs, fresh ...",9,1,[bananas],[]
10,berry good sandwich spread,109439,5,49168,2005-01-25,"[15-minutes-or-less, time-to-make, course, pre...","[79.2, 3.0, 58.0, 0.0, 0.0, 6.0, 5.0]",2,"[in medium size bowl , coarsely mash cranberry...",horseradish is one of my favorite condiments a...,"[whole berry cranberry sauce, sour cream, prep...",3,0,[],[]
11,better than sex strawberries,42198,1460,41531,2002-10-03,"[weeknight, time-to-make, course, main-ingredi...","[734.1, 66.0, 199.0, 10.0, 10.0, 117.0, 28.0]",8,[crush vanilla wafers into fine crumbs and lin...,simple but sexy. this was in my local newspape...,"[vanilla wafers, butter, powdered sugar, eggs,...",7,0,[],[]
